In [14]:
line="{{product_name:浙江在线杭州}}{{time:4月25日}}讯（记者{{person_name: 施宇翔}} 通讯员 {{person_name:方英}}）毒贩很“时髦”，用{{product_name:微信}}交易毒品。没料想警方也很“潮”，将计就计，一举将其擒获。记者从{{org_name:杭州江干区公安分局}}了解到，经过一个多月的侦查工作，{{org_name:江干区禁毒专案组}}抓获吸贩毒人员5名，缴获“冰毒”400余克，毒资30000余元，扣押汽车一辆。{{location:黑龙江}}籍男子{{person_name:钱某}}长期落脚于宾馆、单身公寓，经常变换住址。他有一辆车，经常半夜驾车来往于{{location:杭州主城区}}的各大宾馆和单身公寓，并且常要活动到{{time:凌晨6、7点钟}}，{{time:白天}}则在家里呼呼大睡。{{person_name:钱某}}不寻常的特征，引起了警方注意。禁毒大队通过侦查，发现{{person_name:钱某}}实际上是在向落脚于宾馆和单身公寓的吸毒人员贩送“冰毒”。"
entity=["time","location","person_name","org_name","company_name","product_name"]
import re,jieba


In [15]:
newline=[]
loc_entity=[]
person_entity=[]
org_entity=[]
doubelsurname=['欧阳', '太史', '端木', '上官', '司马', '东方', '独孤', '南宫', '万俟', '闻人', '夏侯', '诸葛', '尉迟', '公羊', '赫连', '澹台', '皇甫', '宗政', '濮阳', '公冶', '太叔', '申屠', '公孙', '慕容', '仲孙', '钟离', '长孙', '宇文', '司徒', '鲜于', '司空', '闾丘', '子车', '亓官', '司寇', '巫马', '公西', '颛孙', '壤驷', '公良', '漆雕', '乐正', '宰父', '谷梁', '拓跋', '夹谷', '轩辕', '令狐', '段干', '百里', '呼延', '东郭', '南门', '羊舌', '微生', '公户', '公玉', '公仪', '梁丘', '公仲', '公上', '公门', '公山', '公坚', '左丘', '公伯', '西门', '公祖', '第五', '公乘', '贯丘', '公皙', '南荣', '东里', '东宫', '仲长', '子书', '子桑', '即墨', '达奚', '褚师', '吴铭']

def append_entity(entity,newline):
    entity=list(jieba.cut(entity))
    for ent in entity:
        newline.append(ent)
    return newline,entity

def perEntity(entity):
    name=[]
    length=len(entity)
    if length!=1:
        if entity[0] not in doubelsurname and len(entity[0])==2:
            name.append("C")
            
            
            
            

compile=re.compile("{{(.*?)}}")
filter_compile=re.compile("\w+:")
for lin in compile.split(line):
    if len(lin)==0:
        continue
    if filter_compile.match(lin):
        entity_name,entity=lin.split(":")
        newline,entity=append_entity(entity,newline)
        if entity_name=='location':
            for index,ent in enumerate(entity):
                
                pass
        elif entity_name=='person_name':
            pass
        elif entity_name=='org_name':
            pass
        elif entity_name=='company_name':
            pass
        else:
            pass
    else:
        newline,_=append_entity(lin,newline)
        

        





Building prefix dict from the default dictionary ...


Dumping model to file cache /var/folders/83/ng05nj_s1_s9842rkp7xn5jc0000gp/T/jieba.cache


Loading model cost 1.737 seconds.


Prefix dict has been built succesfully.


['黑龙江']
['杭州', '主城区']


In [28]:
list(jieba.cut("张东北男黑红姐"))

['张', '东北', '男', '黑红', '姐']

In [16]:
from queue import LifoQueue,PriorityQueue,Queue
from threading import Thread
import time
class ThreadPoolManger():
    """线程池管理器"""
    def __init__(self, thread_num):
        # 初始化参数
        self.work_queue = Queue()
        self.thread_num = thread_num
        self.__init_threading_pool(self.thread_num)

    def __init_threading_pool(self, thread_num):
        self.ts=[]
        # 初始化线程池，创建指定数量的线程池
        for i in range(thread_num):
            thread = ThreadManger(self.work_queue)
            thread.start()
            self.ts.append(thread)
    def add_job(self, func, *args):
        # 将任务放入队列，等待线程池阻塞读取，参数是被执行的函数和函数的参数
        self.work_queue.put((func, args))
    
    def join(self):
        for t in self.ts:
            t.join()
            
class ThreadManger(Thread):
    """定义线程类，继承threading.Thread"""
    def __init__(self, work_queue):
        Thread.__init__(self)
        self.work_queue = work_queue
        self.daemon = True

    def run(self):
        # 启动线程
        while True:
            target, args = self.work_queue.get()
            if target=="stop":
                break
            target(*args)
            self.work_queue.task_done()

import random
queue=LifoQueue()
def aa(i):
    print("A",i)
    queue.put(i)
    time.sleep(random.uniform(0,1))


def a():
    thread_pool = ThreadPoolManger(3)
    for i in range(16):
        thread_pool.add_job(aa,*(i,))
        
    thread_pool.add_job("stop",*(0,))
    thread_pool.add_job("stop",*(0,))
    thread_pool.add_job("stop",*(0,))

    thread_pool.join()
    print("AAAAAAAAAAAA")
    queue.put('a')
    
def b():
    ab=True
    while ab:
        a=queue.get()
        if (a=='a' and queue.empty()) or queue.empty():
            print(a)
            break
        print(a)
        time.sleep(0.5)
ts=[]

time.sleep(1)
ts.append(Thread(target=a))
ts[-1].start()
time.sleep(1)
ts.append(Thread(target=b))
ts[-1].start()

for t in ts:
    t.join()
    

A 0
A 1
A 2


A 3
A 4


A 5
A 6


A 7
7


A 8
A 9


...

4


3


2


1


0
